# Projeto: Aplicação Web de Visualização de Localizações com APIs


## 🌍 Parte 1: Aquisição de Dados da API

Este script Python é responsável por popular a base de dados opengeo.sqlite. Ele lê endereços de um ficheiro where.data e, para cada endereço, faz uma chamada à API de geolocalização. Os resultados JSON da API são armazenados na tabela Locations da base de dados. **Muito importante: Os ficheiros where.data e o opengeo.sqlite contêm material proprietário do curso e, por isso, não podem ser incluídos neste repositório público. O script é apresentado aqui para demonstrar a lógica de aquisição e armazenamento de dados.**

In [48]:
import urllib.request,urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl 
import sys

serviceurl = "https://py4e-data.dr-chuck.net/opengeo?"

conn = sqlite3.connect ("opengeo.sqlite")
cur = conn.cursor ()

cur.execute ("""
CREATE TABLE IF NOT EXISTS Locations (adress TEXT, geodata TEXT
)
""")

ctx = ssl.create_default_context ()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open (r"C:\Users\iness\py4e\opengeo\exemplo_where.data.txt.txt", "r")
count = 0
notfound = 0
for line in fh:
    if count > 100:
        print ("Retrieved 100 locations, resart to retrieve more")
        break
    adress = line.strip ()
    print (" ")
    cur.execute ("SELECT geodata FROM Locations WHERE adress = ?",(adress,))
    
    try: 
        data = cur.fetchone ()[0]
        print ("Found in db", adress)
        continue
    except:
        pass
    parms = dict ()
    parms ["q"] = adress
    url = serviceurl + urllib.parse.urlencode(parms)
    
    print ("Retrieving",url)
    uh = urllib.request.urlopen (url,context = ctx)
    data = uh.read ().decode ()
    print ("Retrieved", len (data), "characters", data [:20].replace ("\n", " "))
    count = count + 1 
    
    try:
        js = json.loads (data)
    except:
        print (data)
        continue
    if not js or "features" not in js:
        print ("==Download error ==")
        print (data)
        break
    if len (js ["features"]) == 0:
        print ("==Objevt not found ==")
        notfound = notfound + 1
        
    cur.execute("""INSERT INTO Locations (adress, geodata) VALUES (?, ?)""", (adress, data))
    conn.commit ()
    
    if count % 10 == 0:
        print ("Pausing")
if notfound >0:
    print ("Number of features for which the location could not be found", notfound)


 
Retrieving https://py4e-data.dr-chuck.net/opengeo?q=Torre+Eiffel%2C+Paris%2C+Fran%C3%83%C2%A7a
Retrieved 1309 characters {"type":"FeatureColl
 
Retrieving https://py4e-data.dr-chuck.net/opengeo?q=Big+Ben%2C+Londres%2C+Reino+Unido
Retrieved 1705 characters {"type":"FeatureColl
 
Retrieving https://py4e-data.dr-chuck.net/opengeo?q=Est%C3%83%C2%A1tua+da+Liberdade%2C+Nova+Iorque%2C+EUA
Retrieved 1363 characters {"type":"FeatureColl


## ⚙️ Parte 2: Processamento de Dados e Geração de Ficheiro para Frontend

Este script lê os dados brutos da base de dados opengeo.sqlite e extrai a latitude, longitude e o endereço formatado dos dados JSON. Ele formata esses dados num array JavaScript (myData) e escreve-o para where.js, que é consumido pela interface web. Este script pode ser executado após a execução do script de aquisição de dados (Parte 1), usando o opengeo.sqlite gerado a partir do exemplo_where.data.

In [50]:
import sqlite3
import json
import codecs

conn = sqlite3.connect ("opengeo.sqlite")
cur = conn.cursor ()

cur.execute ("SELECT * FROM Locations")
fhnd = codecs.open ("where.js", "w", "utf-8")
fhnd.write ("myData = [\n")
count = 0
for row in cur:
    data = str (row[1])
    try: 
        js = json.loads (str(data))
    except:
        continue
    if len (js ["features"]) == 0:
        continue
        
    try:
        lat = js ["features"][0]["geometry"]["coordinates"][1]
        lng = js ["features"][0]["geometry"]["coordinates"][0]
        where = js ["features"][0]["properties"]["formatted"]
        where = where.replace ("'", " ")
    except:
        print ("Unexpected format")
        print (js)
    try:
        print (where, lat, lng)
        count = count + 1 
        if count >1:
            fhnd.write (",\n")
            output = "["+str(lat)+","+str(lng)+", '"+where+"']"
            fhnd.write (output)
    except: 
        continue
fhnd.write("\n];\n")
cur.close()
fhnd.close()

Torre Eiffel, 28859 Torrejón De Ardoz, Spain 40.439687 -3.459808
Big Ben Fisheries, 9-13 Broadway, London, W13 9DA, United Kingdom 51.5103183 -0.3225142
New York, NY, United States of America 40.68295 -73.9708


In [40]:
import os

print(f"Existe 'opengeo.sqlite'? {os.path.exists('opengeo.sqlite')}")
print(f"Existe 'where.js'? {os.path.exists('where.js')}")
print(f"Conteúdo da pasta atual: {os.listdir('.')}")

Existe 'opengeo.sqlite'? True
Existe 'where.js'? True
Conteúdo da pasta atual: ['.anaconda', '.conda', '.condarc', '.continuum', '.idlerc', '.ipynb_checkpoints', '.ipython', '.jme3', '.jupyter', '.matplotlib', '.Origin', '.poreidcache', '.QtWebEngineProcess', '.spss', '.virtual_documents', '3-1.2ExcecptionHandling.ipynb', '3D Objects', 'amd.json', 'anaconda3', 'anaconda_projects', 'AppData', 'apple.json', 'Application Data', 'bamboo_train_tree.dot', 'bamboo_tree.dot', 'BrawlhallaReplays', 'Contacts', 'Cookies', 'Creative Cloud Files Personal Account inessantos2001@gmail.com 898374095E95BDFE0A495FD5@AdobeID', 'DataScienceEcosystem.ipynb.ipynb', 'Definições Locais', 'Documents', 'dog.jpg', 'Downloads', 'Dropbox', 'DS0103EN-Exercise-From-Modeling-to-Evaluation.ipynb', 'DS0103EN-Exercise-From-Understanding-to-Preparation.ipynb', 'DS0103EN-Review-From-Requirements-to-Collection (1).ipynb', 'EEG.ipynb', 'Evernote', 'example.txt', 'example1.txt', 'Example2.txt', 'Example3.txt', 'Favorites', '